In [54]:
import xml.etree.ElementTree as ET
import pandas as pd

In [55]:
# Parse the XML file
tree = ET.parse('xml/20230712/PV-9-2023-07-12-VOT_EN.xml')

# Get the root element
root = tree.getroot()

nature_restoration = root.find('.//Vote.Result[@Number="6."]')

In [56]:
# print(nature_restoration)

# for child in nature_restoration:
#     print(nature_restoration.get("Vote.Result.Text.Title"))
#     print(child.tag, child.attrib)

text_element = nature_restoration.find('Vote.Result.Text.Title')
if text_element is not None:
    print(text_element.text)

results_element = nature_restoration.find("Vote.Result.Table.Results")

Nature restoration ***I


In [57]:
# Initialize lists to store data
data = []

# Find all <TR> elements
for tr in results_element.findall('.//TR'):
    # Initialize a dictionary to store row data
    row_data = {}

    # Iterate over <TD> elements within the <TR> element
    for td in tr.findall('TD'):
        p_elem = td.find('P')

        if hasattr(p_elem, "text"):
            text = p_elem.text.strip()

            # Get the column name from the 'COLNAME' attribute of <TD>
            colname = td.get('COLNAME')
            
            match colname:
                case "C1":
                    text = text.replace("§", "paragraph")
                case "C5":
                    match text:
                        case "+":
                            text = "adopted"
                        case "—":
                            text = "rejected"
                        case "↓":
                            text = "lapsed"
                case 'C6':
                    vote_for = p_elem.find('Vote.Result.Table.TotalVote.For')
                    vote_against = p_elem.find('Vote.Result.Table.TotalVote.Against')
                    vote_abstension = p_elem.find('Vote.Result.Table.TotalVote.Abstention')
    
                    if vote_for is not None and vote_against is not None and vote_abstension is not None:
                        if hasattr(vote_for, "text") and hasattr(vote_against, "text") and hasattr(vote_abstension, "text"):
                            text = vote_for.text + ", " + vote_against.text + ", " + vote_abstension.text

            # Add data to the row dictionary
            row_data[colname] = text

    # Append the row data to the list of data
    data.append(row_data)

#Create a DataFrame from the list of data
df = pd.DataFrame(data)

# Set the first column of the dataframe as the header
df.columns = df.iloc[0]
df = df[1:]
df = df.reset_index(drop=True)

# Display the DataFrame
df

,Subject,Am No,Author,RCV etc.,Vote,RCV/EV – remarks
0,Proposal to reject the Commission proposal,NaN,NaN,NaN,NaN,NaN
1,Rejection,NaN,committee,RCV,rejected,"312, 324, 12\n ..."
2,Draft legislative act,NaN,NaN,NaN,NaN,NaN
3,Text as a whole,18CP,Renew,EV,adopted,"371, 223, 53\n ..."
4,"Article 1, paragraph 1, point a",113,MEPs,EV,adopted,"347, 298, 7\n ..."
...,...,...,...,...,...,...
158,Recital 57,112D,MEPs,EV,adopted,"333, 314, 3\n ..."
159,NaN,18CP,Renew,NaN,lapsed,NaN
160,After recital 62,56,The Left,NaN,rejected,NaN
161,After recital 71,57,The Left,NaN,rejected,NaN


In [58]:
# Replace the first column empty row with previous row
for index, row in df.iterrows():
    if pd.isna(row['Subject']):
        # Replace empty cell with the value from the previous row
        df.at[index, 'Subject'] = df.at[index - 1, 'Subject']

df

,Subject,Am No,Author,RCV etc.,Vote,RCV/EV – remarks
0,Proposal to reject the Commission proposal,NaN,NaN,NaN,NaN,NaN
1,Rejection,NaN,committee,RCV,rejected,"312, 324, 12\n ..."
2,Draft legislative act,NaN,NaN,NaN,NaN,NaN
3,Text as a whole,18CP,Renew,EV,adopted,"371, 223, 53\n ..."
4,"Article 1, paragraph 1, point a",113,MEPs,EV,adopted,"347, 298, 7\n ..."
...,...,...,...,...,...,...
158,Recital 57,112D,MEPs,EV,adopted,"333, 314, 3\n ..."
159,Recital 57,18CP,Renew,NaN,lapsed,NaN
160,After recital 62,56,The Left,NaN,rejected,NaN
161,After recital 71,57,The Left,NaN,rejected,NaN


### Split votes column into three columns

In [59]:
# Split the values in the 'values' column by ","
df[['for', 'against', 'abstension']] = df['RCV/EV – remarks'].str.split(', ', expand=True)

# Convert the columns to numeric if needed
df['for'] = pd.to_numeric(df['for'])
df['for'] = df['for'].astype(pd.Int64Dtype())
    
df['against'] = pd.to_numeric(df['against'], downcast='integer')
df['against'] = df['against'].astype(pd.Int64Dtype())

df['abstension'] = pd.to_numeric(df['abstension'], downcast='integer')
df['abstension'] = df['abstension'].astype(pd.Int64Dtype())

# Drop "RCV/EV – remarks" column
df.drop(columns=['RCV/EV – remarks'], inplace=True)

df

,Subject,Am No,Author,RCV etc.,Vote,for,against,abstension
0,Proposal to reject the Commission proposal,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>
1,Rejection,NaN,committee,RCV,rejected,312,324,12
2,Draft legislative act,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>
3,Text as a whole,18CP,Renew,EV,adopted,371,223,53
4,"Article 1, paragraph 1, point a",113,MEPs,EV,adopted,347,298,7
...,...,...,...,...,...,...,...,...
158,Recital 57,112D,MEPs,EV,adopted,333,314,3
159,Recital 57,18CP,Renew,NaN,lapsed,<NA>,<NA>,<NA>
160,After recital 62,56,The Left,NaN,rejected,<NA>,<NA>,<NA>
161,After recital 71,57,The Left,NaN,rejected,<NA>,<NA>,<NA>


In [60]:
# df.to_csv('C:/Users/sc/PycharmProjects/testRCVScraping/xml/20230712/test.csv', index=False)